In [1]:
import torch
import os
import random
import numpy as np
import pytorch_lightning as pl

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from dataset.utils import process_classification_dataset

config = {
        'dataset': 'facebook_ct1', # 'infectious_ct1', #
        'hidden_size': 32,
        'rnn_layers': 8,
        'readout_layers': 3,
        'cell_type': 'lstm',
        'dim_red': 64,
        'add_self_loops': False,
        'verbose': True,
        'cat_states_layers': True,
        'weight_decay': 1e-3,
        'step_size': 30,
        'gamma': 0.5
        }

# Select one GPU if more are available
os.environ["CUDA_VISIBLE_DEVICES"]='0'

seed = 42
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
pl.seed_everything(seed)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

Seed set to 42


In [2]:
# Fix cuda memory issue
dataset, states, node_states, node_labels = process_classification_dataset(config, "DynCRNN", device)

100%|██████████| 995/995 [11:23<00:00,  1.46it/s]


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

inputs, test_inputs, labels, test_labels = train_test_split(states.inputs[:,-1,:], dataset.targets, test_size=0.2, random_state=seed)

# Create the classifier
classifier = LogisticRegression(max_iter=5000, random_state=seed)

# Train the classifier
classifier.fit(inputs, labels)

# Test the classifier
y_pred = classifier.predict(test_inputs)

accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9547738693467337


# PCA per node

In [4]:
from sklearn.decomposition import PCA
from einops import rearrange

node_states0 = rearrange(node_states[0], 't n f -> n t f')

pca = PCA(n_components=10)
X_red = pca.fit_transform(rearrange(node_states0, 'n t f -> (n t) f'))

In [5]:
from DMD.sindy import sindy_library, sindy_fit, sindy_fit2

s = rearrange(X_red, '(n t) f -> t (n f)', n=node_states0.shape[0], t=node_states0.shape[1], f=10)
s = sindy_library(s, 2)
s = rearrange(s, 't (n f) -> n t f', n=node_states0.shape[0], t=node_states0.shape[1])

Xi = sindy_fit2(s[1:,:], s[:-1,:])

EinopsError:  Error while processing rearrange-reduction pattern "t (n f) -> n t f".
 Input tensor shape: (106, 501501). Additional info: {'n': 100, 't': 106}.
 Shape mismatch, can't divide axis of length 501501 in chunks of 100